In [1]:
from astropy.cosmology import FlatLambdaCDM
import numpy as np

In [2]:
data = np.loadtxt('data/C11.csv', delimiter=',')

In [4]:
data[:,0]

array([0.50349, 0.4952 , 0.6782 , ..., 1.014  , 1.315  , 1.092  ])

In [6]:
cosmo = FlatLambdaCDM(H0=70, Om0 = 0.3)

In [8]:
length = cosmo.luminosity_distance(data[:, 0])

In [11]:
uth = 5 * np.log10(length.value) + 25

In [12]:
A = ((uth - data[:, 1]) ** 2 / data[:,2]).sum()

In [13]:
A

149.91787444475813

In [16]:
uth[-2]

44.835882460703445

In [15]:
 data[:, 1][-1]

43.9397

In [17]:
data[:,2]

array([0.1337, 0.118 , 0.2142, ..., 0.2433, 0.2001, 0.28  ])

In [23]:
cosmo.luminosity_distance(1e-5) * 70 / 3e5 / (1 + 1e-5)

<Quantity 9.99305945e-06 Mpc>